# Flavor Frontier Analysts 

Import Dependencies

In [1]:
import pandas as pd
from pymongo import MongoClient
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import json
import webbrowser
import os


Initialize Flask App and Set Up CORS

In [ ]:
app = Flask(__name__)
CORS(app)

Connect to MongoDB and Create a Database and Collection

In [54]:
client = MongoClient(port=27017)
db = client['yelp_database']
collection = db['businesses']

Import the Yelp data .json 

In [55]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)

Covnert the categories field to string and then filter for results that include the string "Restaurants" in the 'categories' column.

In [57]:
df['categories'] = df['categories'].astype(str)
df_restaurants = df[df['categories'].str.contains('Restaurants', case=False, na=False)]

Create two lists from strings in the 'categories' column. One list of categories that are strictly restaurants, and another for food related categories. 
These lists will be passed to serve as the user input options as well as the main query variable.

In [63]:
strict_restaurant_categories = [
    "Restaurants", "American (Traditional)", "American (New)", "Italian", 
    "Mexican", "Chinese", "Japanese", "Thai", "Vietnamese", "Indian", "Korean", 
    "Mediterranean", "Greek", "French", "Spanish", "German", "Irish", "British", 
    "Seafood", "Steakhouses", "Pizza", "Burgers", "Sushi Bars", 
    "Barbeque", "Asian Fusion", "Diners", "Breakfast & Brunch", 
    "Buffets", "Comfort Food", "Soul Food", "Southern", 
    "Tex-Mex", "Cajun/Creole", "Latin American", "Middle Eastern", "African", 
    "Caribbean", "Brazilian", "Peruvian", "Cuban", "Tapas Bars", "Gastropubs",
    "Izakaya", "Ramen", "Poke", "Hot Pot", "Dim Sum", "Fondue",
    "Fish & Chips", "Bistros", "Brasseries",
    "Donairs", "Kebab", "Falafel", "Tacos", "Cheesesteaks"
]


food_related_categories = [
    "Food", "Sandwiches", "Cafes", "Fast Food", "Pubs", 
    "Halal", "Kosher", "Vegan", "Vegetarian",
    "Gluten-Free", "Organic", "Farm-to-table", "Food Trucks", "Food Stands",
    "Creperies", "Delis", "Noodles", "Soup", "Salad", "Chicken Wings",
    "Chicken Shop", "Hot Dogs", "Waffles", "Pancakes", "Donuts", "Bagels",
    "Food", "Specialty Food", "Caterers", "Bakeries", "Desserts", "Juice Bars & Smoothies",
    "Ice Cream & Frozen Yogurt", "Gelato", "Coffee & Tea"
]

Created cleaned dataframe with only relevent columns

In [ ]:
df_restaurants_clean = df_restaurants[['business_id', 'name', 'city', 'state', 'latitude', 'longitude', 'categories', 'stars']]

## Delete any existing data from previous runs and setup Mongo

In [65]:
collection.delete_many({})

In [ ]:
if collection.count_documents({}) == 0:
    records = df_restaurants_clean.to_dict('records')
    collection.insert_many(records)
    print("Data loaded into MongoDB successfully.")
else:
    print("Data already exists in MongoDB.")

## Setup flask api routes

Categories route

In [67]:
@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify({
        'strict_restaurant_categories': strict_restaurant_categories
    })


Query restaurant route

In [68]:

@app.route('/query_restaurants/<category>', methods=['GET'])
def query_restaurants(category):
    query = {"categories": {'$regex': category, '$options': 'i'}}

    results = list(collection.find(query))

    # Convert ObjectId to string for JSON serialization <------- important.
    for result in results:
        result['_id'] = str(result['_id'])

    return jsonify(results)


Serve the .html and the 

In [69]:
base_dir = os.path.dirname(os.path.abspath(__file__))
# Serve HTML file
@app.route('/')
def serve_html():
    return send_from_directory(base_dir, 'index.html')

# Serve JavaScript file
@app.route('/<path:path>')
def serve_file(path):
    return send_from_directory(base_dir, path)

Open the web browser

In [ ]:
webbrowser.open('http://localhost:5000/')
if __name__ == '__main__':
    app.run(debug=True)
